组归一化-Grop Normalization
===

归一化策略主要有四种，分别是Batch Normalization，Layer Normalization，Instance Normalization以及Group Normalization。四种归一化策略可以用下图来表示

![Images](Images/03/00/08_01_001.jpg)

N表示样本轴，C表示通道轴，F是每个通道的特征数量。

1. BN归一化是多个样本的统一通道，所以适用于CNN以及batchsize较大的情况。BN计算归一化统计量时考虑了一个批量中所有图片的内容，从而造成了每个样本独特细节的丢失。
2. LN归一化是同一样本的不同通道，所以适用于动态网络以及batchsize较小的网络。LN计算时考虑一个样本所有通道，从而忽略了不同通道的差异
3. IN适合于图像风格迁移这类应用，因为它计算归一化统计量时考虑单个样本，单个通道的所有元素。
4. GN适用的范围最广，因为它计算归一化统计量时考虑单个样本，多个通道的所有元素。它的效果最好

# 1.批归一化的问题

批归一化(Batch Norm/BN)是深度学习中非常有效的一个技术，极大地推进了计算机视觉以及之外领域的前沿。BN通过计算一个(迷你)批量中的均值与方差来进行特征归一化。众多实践证明，它利于优化且使得深度网络易于收敛。批统计的随机不确定性也作为一个有利于泛化的正则化项。BN 已经成为了许多顶级计算机视觉算法的基础

尽管取得了很大的成果，BN也会因为归一不同批尺寸的独特行为而有缺点。特别是，BN需要用到足够大的批大小(例如，每个工作站采用32的批量大小)。一个小批量会导致估算批统计不准确，减小BN的批大小会极大地增加模型错误率。结果导致，如今许多模型都使用较大的批训练，它们非常耗费内存。反过来，训练模型时对BN效力的极度依赖性阻碍了人们用有限内存探索更高容量的模型。
![Images](Images/03/00/08_01_002.jpg)

# 2.四种归一化的比较

各种归一化算法都是GN的特殊情况。对于GN来说，是根据盖层的输入数据计算均值和方差，然后使用这两个值更新输入数据，公式如下：

$$
\begin{split}
\mu_i&=\frac{1}{m}\sum_{k \in S_i}x_k \\
\sigma_i&=\sqrt{\frac{1}{m}\sum_{k \in S_i}(x_k-\mu_i)^2+\epsilon} \\
\hat{x_i}&=\frac{1}{\sigma_i}(x_i-\mu_i)
\end{split}
$$

对于BN来说，它是取不同batch的同一channel上的所有的值，有
$$S_i=\{k|k_C=i_C\}$$

对于LN来说，则是从同一个batch的不同channel上取所有的值，有
$$S_i=\{k|k_N=i_N\}$$

对于IN来说，即不跨batch，也不跨channel，有
$$S_i=\{k=k_N=i_N,k_C=i_C\}$$

对于GN来说，将Channel分为若干组，只是用组内的数据计算平均值和方差，通常组数G是一个超参数，有
$$S_i=\{k|k_n=i_N,\lfloor \frac{k_C}{\frac{C}{G}}\rfloor=\lfloor \frac{i_C}{\frac{C}{G}} \rfloor\}$$

我们可以看出，当GN的组数为1是，此时GN与LN等价；当GN的组数为通道数时，GN与IN等价

# 3.组归一化

In [ ]:
import torch
import torch.nn as nn

class GroupNorm(nn.Module):
    def __init__(self, num_features, num_groups=32, eps=1e-5):
        super(GroupNorm, self).__init__()
        self.weight = nn.Parameter(torch.ones(1,num_features,1,1))
        self.bias = nn.Parameter(torch.zeros(1,num_features,1,1))
        self.num_groups = num_groups
        self.eps = eps

    def forward(self, x):
        N,C,H,W = x.size()
        G = self.num_groups
        assert C % G == 0

        x = x.view(N,G,-1)
        mean = x.mean(-1, keepdim=True)
        var = x.var(-1, keepdim=True)

        x = (x-mean) / (var+self.eps).sqrt()
        x = x.view(N,C,H,W)
        return x * self.weight + self.bias